In [33]:
# import packages
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA,TruncatedSVD

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, LSTM, Embedding, SpatialDropout1D, Lambda, Reshape, Conv1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow_hub as hub

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

import time

import gc

from numba import cuda

In [34]:
from cuml.naive_bayes import MultinomialNB
from cuml.ensemble import RandomForestClassifier as cuRFC

In [35]:
import cudf, cuml
from cuml.neighbors import KNeighborsClassifier as cuKNeighbors
from cuml.ensemble import RandomForestClassifier as cumlRandomForestClassifier

In [36]:
import cupy as cp
from cuml.svm import SVC

In [37]:
#num_sample = 5000
num_sample = 100000

In [38]:
results = []

In [39]:
df = pd.read_csv('/kaggle/input/686-project/df_new.csv').sample(num_sample,random_state = 1234)
decode_map = {"negative":0, "positive":1}
df.target = df.target.apply(lambda x: decode_map[x])
# split train and test data
X_train, X_test, y_train, y_test = train_test_split(df['text'].astype(str), df['target'],test_size=0.20,
                                                           random_state=1234,
                                                           stratify = df['target'])

In [40]:
%%time
# consider both unigrams and bigrams in vectorizer
#tokenizer = TfidfVectorizer(min_df = 0.0001, ngram_range = (1,2))
tokenizer = TfidfVectorizer(min_df = 0.001, ngram_range = (1,2))
tokenizer.fit(X_train)
X_train = tokenizer.transform(X_train)
X_test = tokenizer.transform(X_test)
X_train.shape

CPU times: user 2.92 s, sys: 90.9 ms, total: 3.01 s
Wall time: 3.01 s


(80000, 1071)

# KNN

In [41]:
%%time
model = cuKNeighbors()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
predictions = model.predict(X_test)
results.append(["KNN",f1_score(y_test,predictions), accuracy_score(y_test,predictions), log_loss(y_test,predictions), end-start])
print("F_1 Score of Model:",f1_score(y_test,predictions))
print("Accuracy of Model:",accuracy_score(y_test,predictions))
print("Cross-entropy loss of Model:",log_loss(y_test,predictions))

F_1 Score of Model: 0.6759835800931692
Accuracy of Model: 0.64875
Cross-entropy loss of Model: 12.131917322059225
CPU times: user 605 ms, sys: 244 ms, total: 849 ms
Wall time: 847 ms


# Naive Bayes

In [42]:
%%time
model = MultinomialNB()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
predictions = model.predict(X_test)
results.append(["Naive Bayes",f1_score(y_test,predictions), accuracy_score(y_test,predictions), log_loss(y_test,predictions), end - start])
print("F_1 Score of Model:",f1_score(y_test,predictions))
print("Accuracy of Model:",accuracy_score(y_test,predictions))
print("Cross-entropy loss of Model:",log_loss(y_test,predictions))

F_1 Score of Model: 0.7405994816879371
Accuracy of Model: 0.73475
Cross-entropy loss of Model: 9.16152358178643
CPU times: user 48.4 ms, sys: 2.93 ms, total: 51.3 ms
Wall time: 49.9 ms


# Random Forest

In [43]:
%%time
model = cuRFC()
start = time.time()
model.fit(X_train.toarray(), y_train)
end = time.time()
predictions = model.predict(X_test.toarray())
results.append(["Random Forest",f1_score(y_test,predictions), accuracy_score(y_test,predictions), log_loss(y_test,predictions),end - start])
print("F_1 Score of Model:",f1_score(y_test,predictions))
print("Accuracy of Model:",accuracy_score(y_test,predictions))
print("Cross-entropy loss of Model:",log_loss(y_test,predictions))

/opt/conda/lib/python3.7/site-packages/cuml/internals/api_decorators.py:567: UserWarning: To use pickling or GPU-based prediction first train using float32 data to fit the estimator
  ret_val = func(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/cuml/internals/api_decorators.py:586: UserWarning: GPU based predict only accepts np.float32 data. The model was trained on np.float64 data hence cannot use GPU-based prediction! 
Defaulting to CPU-based Prediction. 
To predict on float-64 data, set parameter predict_model = 'CPU'
  ret_val = func(*args, **kwargs)


F_1 Score of Model: 0.7350712726338771
Accuracy of Model: 0.69055
Cross-entropy loss of Model: 10.688213340257738
CPU times: user 6.49 s, sys: 1.14 s, total: 7.62 s
Wall time: 5.33 s


# SVM

In [44]:
%%time
model = SVC()
start = time.time()
model.fit(X_train.toarray(), y_train)
end = time.time()
predictions = model.predict(X_test.toarray())
results.append(["SVM",f1_score(y_test,predictions), accuracy_score(y_test,predictions), log_loss(y_test,predictions), end - start])
print("F_1 Score of Model:",f1_score(y_test,predictions))
print("Accuracy of Model:",accuracy_score(y_test,predictions))
print("Cross-entropy loss of Model:",log_loss(y_test,predictions))

F_1 Score of Model: 0.7524866657056365
Accuracy of Model: 0.74245
Cross-entropy loss of Model: 8.895579121472384


# Xgboost

In [45]:
%%time
xgb = XGBClassifier(tree_method='gpu_hist')
start = time.time()
xgb.fit(X_train,y_train)
end = time.time()
predictions = xgb.predict(X_test)
results.append(["Xgboost",f1_score(y_test,predictions), accuracy_score(y_test,predictions), log_loss(y_test,predictions), end - start])
print("F_1 Score of Model:",f1_score(y_test,predictions))
print("Accuracy of Model:",accuracy_score(y_test,predictions))
print("Cross-entropy loss of Model:",log_loss(y_test,predictions))

F_1 Score of Model: 0.7482636493261581
Accuracy of Model: 0.72635
Cross-entropy loss of Model: 9.45167844883001
CPU times: user 1.27 s, sys: 7.95 ms, total: 1.28 s
Wall time: 1.19 s


In [46]:
results = np.array(results)
result = pd.DataFrame(np.round(results[:,1:].astype(float),3), index = results[:,0], columns = ['F-1','Accuracy','Log-loss','training time'])
print(result)

                 F-1  Accuracy  Log-loss  training time
KNN            0.676     0.649    12.132          0.016
Naive Bayes    0.741     0.735     9.162          0.017
Random Forest  0.735     0.691    10.688          3.762
SVM            0.752     0.742     8.896         18.925
Xgboost        0.748     0.726     9.452          1.117


In [47]:
break

SyntaxError: 'break' outside loop (668683560.py, line 4)

In [ ]:
df_sentence = pd.read_csv('/kaggle/input/686-project/df_new.csv')
decode_map = {"negative":0, "positive":1}
df_sentence.target = df_sentence.target.apply(lambda x: decode_map[x])
df_sentence = df_sentence.sample(num_sample,random_state = 1234)
label = np.array(pd.get_dummies(df_sentence.target), dtype=int)[:] 
# split train and test data
X_train_sen, X_test_sen, y_train_sen, y_test_sen = train_test_split(df_sentence['text'].astype(str), label,test_size=0.20,
                                                           random_state=1234,
                                                           stratify = label)

In [ ]:
print("Training on GPU...") if tf.test.is_gpu_available() else print("Training on CPU...")
embed = "https://tfhub.dev/google/universal-sentence-encoder/4"
hub_layer = hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=False)
model = keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Reshape((512,1)))
model.add(LSTM(32))
model.add(Dense(10))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
batch_size = 128
model.fit(X_train_sen, y_train_sen, epochs = 20, batch_size=batch_size, verbose = 2, shuffle=False)

In [ ]:
predictions = (np.ones((X_test_sen.shape[0],2))* (model.predict(X_test_sen)>0.5))[:,1]
y_test_sen = y_test_sen[:,1]
results.append(["LSTM",f1_score(y_test_sen,predictions), accuracy_score(y_test_sen,predictions), log_loss(y_test_sen,predictions)])
print("F_1 Score of Model:",f1_score(y_test_sen,predictions))
print("Accuracy of Model:",accuracy_score(y_test_sen,predictions))
print("Cross-entropy loss of Model:",log_loss(y_test_sen,predictions))

In [ ]:
results = np.array(results)
result = pd.DataFrame(np.round(results[:,1:].astype(float),3), index = results[:,0], columns = ['F-1','Accuracy','Log-loss'])
print(result)